In [6]:
import pandas as pd
import pymysql
import matplotlib.pyplot as plt

In [ ]:
# 測試資料庫連線

db_config = {
    "host": "111.235.253.145",
    "user": "root",
    "password": "Aji3cl3qup*",  
    "database": "E.A.T",
    "port": 3307  
}

try:
    # 建立連線
    connection = pymysql.connect(**db_config)
    print("Connection successful!")
    
    # 測試查詢
    with connection.cursor() as cursor:
        cursor.execute("SELECT DATABASE();")
        result = cursor.fetchone()
        print(f"Connected to database: {result[0]}")
    
    connection.close()
except pymysql.MySQLError as e:
    print(f"Failed to connect to database: {e}")

In [ ]:
C:/Users/Administrator/Downloads/PortableGit/CSV/FINAL_SEL/FINAL_SEL_list.csv

In [22]:
from pymysql import ProgrammingError

In [37]:
def load_fuzzy_list_to_db(csv_path, db_config):
    try:
        # 連接 MariaDB
        conn = pymysql.connect(**db_config)
        cursor = conn.cursor()

        # 建立 vegetable_aliases 表（如果尚未存在）
        create_table_query = """
        CREATE TABLE IF NOT EXISTS vegetable_aliases (
            id INT AUTO_INCREMENT PRIMARY KEY,
            fuzzy_name VARCHAR(255) NOT NULL,
            alias_name VARCHAR(255) NOT NULL            
        );
        """
        cursor.execute(create_table_query)

        # 從 CSV 文件中讀取數據
        df = pd.read_csv(csv_path, encoding='utf-8')

        # 檢查數據是否為兩列並轉換為列表
        if df.shape[1] == 2:
            aliases = df.dropna().values.tolist()
        else:
            raise ValueError("CSV 文件必須包含兩列：fuzzy_name 和 alias_name")
        
        # 檢查是否有要插入的數據
        if not aliases:
            raise ValueError("CSV 文件中沒有有效的數據")

        # 將數據插入 vegetable_aliases 表，確保字符串值用引號包圍
        insert_query = "INSERT INTO vegetable_aliases (fuzzy_name, alias_name) VALUES (%s, %s)"
        cursor.executemany(insert_query, [(str(fuzzy), str(alias)) for fuzzy, alias in aliases])
        conn.commit()

    except ProgrammingError as e:
        print(f"SQL 語法錯誤: {e}")
    except Exception as e:
        print(f"發生錯誤: {e}")
    finally:
        if 'cursor' in locals() and cursor:
            cursor.close()
        if 'conn' in locals() and conn:
            conn.close()

# 資料庫配置
db_config = {
    "host": "111.235.253.145",
    "user": "root",
    "password": "Aji3cl3qup*",  
    "database": "E.A.T",
    "port": 3307  
}

# 路徑和操作
csv_path = 'C:/Users/Administrator/Downloads/PortableGit/CSV/FINAL_SEL/FINAL_SEL_list.csv'

# 執行步驟
load_fuzzy_list_to_db(csv_path, db_config)